<a href="https://colab.research.google.com/github/rawanelzehery/Recommender-Systems-Project-/blob/main/Dashbord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash-bootstrap-components
!pip install pycountry

# Import Libraries

In [ ]:
import plotly.express as px
from dash import Dash  , html , dcc
from jupyter_dash import JupyterDash
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dash.dependencies import Input, Output , State
import dash
import dash_bootstrap_components as dbc
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import dash_table
from dash_table import DataTable


C:\Users\Laptop Shop\AppData\Local\Temp\ipykernel_15780\694232062.py:12: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


# Import Dataset

In [ ]:
# Read CSV file into a dataframe
movies_df = pd.read_csv('Recommender_dataset/movies.csv')
ratings_df = pd.read_csv('Recommender_dataset/ratings.csv')
predictions_df = pd.read_csv('Recommender_dataset/all_predictions.csv')


# Similarity model (item - item)

In [ ]:
merged = pd.merge(movies_df, ratings_df, on='movieId')
#-------------------------------
merged = merged.drop(['title', 'userId', 'rating', 'timestamp'], axis=1)
#-------------------------------
genres = merged['genres'].str.get_dummies('|')
#-------------------------------
genres = genres.drop('(no genres listed)', axis=1)
merged = pd.concat([merged, genres], axis=1).drop('genres', axis=1)
#----------------------------
merged = merged.drop_duplicates()
merged = merged.set_index('movieId')

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,
1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
193583,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
193585,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import pdist, squareform

jaccard_distances = pdist(merged.values, metric='jaccard')
#-------------------------------
square_jaccard_distances = squareform(jaccard_distances)
#-------------------------------
jaccard_similarity_array = 1 - square_jaccard_distances
#-------------------------------
distance_df = pd.DataFrame(jaccard_similarity_array,
index=merged.index,
columns=merged.index)
#-------------------------------

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.6,0.166667,0.142857,0.200000,0.0,0.166667,0.400000,0.0,0.142857,...,0.285714,0.166667,0.166667,0.2,0.0,0.500000,0.600000,0.000000,0.166667,0.200000
2,0.600000,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.666667,0.0,0.200000,...,0.000000,0.000000,0.000000,0.0,0.0,0.166667,0.200000,0.000000,0.000000,0.000000
3,0.166667,0.0,1.000000,0.666667,0.500000,0.0,1.000000,0.000000,0.0,0.000000,...,0.200000,0.000000,0.333333,0.0,0.0,0.200000,0.250000,0.000000,0.000000,0.500000
4,0.142857,0.0,0.666667,1.000000,0.333333,0.0,0.666667,0.000000,0.0,0.000000,...,0.166667,0.250000,0.666667,0.0,0.0,0.166667,0.200000,0.333333,0.000000,0.333333
5,0.200000,0.0,0.500000,0.333333,1.000000,0.0,0.500000,0.000000,0.0,0.000000,...,0.250000,0.000000,0.500000,0.0,0.0,0.250000,0.333333,0.000000,0.000000,1.000000


In [ ]:
def get_similar_movies(movie_title, k):
    movie_id = movies_df.loc[movies_df['title'] == movie_title, 'movieId'].iloc[0]
    movie_similarity_scores = distance_df[movie_id].sort_values(ascending=False)
    movie_indices = movie_similarity_scores.index[1:1+int(k)]
    top_k_movies = movies_df[movies_df['movieId'].isin(movie_indices)]
    movie_ratings = []
    for index, row in top_k_movies.iterrows():
        movie_rating = ratings_df.loc[ratings_df['movieId'] == row['movieId'], 'rating'].mean()
        movie_ratings.append(movie_rating)
    top_k_movies['average_rating'] = movie_ratings

    return top_k_movies

# Dashboard

In [ ]:
FONT_AWESOME = "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css"
BOOT_STRAP = "assets\bootstrap.min.css"
app = JupyterDash(external_stylesheets = [BOOT_STRAP,FONT_AWESOME])

In [ ]:
PLOTLY_LOGO = "https://www.svgrepo.com/show/35001/github.svg"
items_per_page = 3
#-----------------------------------------------------------------------------search bar----------------------------------------------------------------------------------------------------------------------
# Define the options for the search bar
search_options = [{'label': str(user_id), 'value': user_id} for user_id in ratings_df['userId'].unique()]
search_Movies = [{'label': title, 'value': title} for title in movies_df['title']]
#-----------------------------------------------------------------------------tabels----------------------------------------------------------------------------------------------------------------------
# Define the columns to display in the table
columns = [
    {'name': 'Title', 'id': 'title'},
    {'name': 'Genres', 'id': 'genres'},
    {'name': 'Your Rating', 'id': 'rating'},
]
columns_movie = [
    {'name': 'Title', 'id': 'title'},
    {'name': 'Genres', 'id': 'genres'},
    {'name': 'Rating', 'id': 'rating'}
]
#------------------------------------------------------------------------------Navbar------------------------------------------------------------------------------------------------
navbar = dbc.Navbar(
    dbc.Container(
        [
            html.A(
                dbc.Row(
                    [
                        dbc.Col(html.Img(src=PLOTLY_LOGO, height="30px")),
                        dbc.Col(dbc.NavbarBrand("Movie", className="ms-2",style={"color" : "#0F7D91","font-size": "40px","font-weight": "bold",'font-family': "'Xanh Mono', monospace"})),
                    ],
                    align="center",
                    className="g-0",
                ),
                href="https://github.com/rawanelzehery/Recommender-Systems-Project-",
                style={"textDecoration": "none"},
            ),
        ]
    ),
    color="dark",
    dark=True,
    style={ 'background':'linear-gradient(to bottom, #B5D3DD, #E1F1F6)'}
)
# #5C86C0, #A4B6D6
#-----------------------------------------------------------------------------Tab1----------------------------------------------------------------------------------------------------------------------
tab1_content = dbc.Container([
    dbc.Row([
        dbc.Col([dbc.Card(
    [
        html.H5(children='LOGIN TO YOUR DASHBOARD',style={"color" : "#0F7D91" , 'font-family': "'Xanh Mono', monospace"}),
        dbc.CardBody([
            dbc.Label(
                dcc.Textarea(
                    className="mb-3",
                    id='search-bar',
                    placeholder='Enter user ID here...',
                    style={'width': '100px'}
                )
            ),
        ]),
         dbc.Button('login', id='search-button', style={'width': '100px','height': '40px' ,"color" : "#FFFFFF" ,"background" : "#0F7D91" , 'font-family': "'Xanh Mono', monospace","font-size": "20px","font-weight": "bold"}),

    ],className='mt-4',style={"height":"13rem","border-width": "medium" ,'background':'linear-gradient(to bottom, #B5D3DD, #E1F1F6)','display': 'flex',
        'justify-content': 'center',
        'align-items': 'center'}
),], width=4),
#-----------------------------------------------------------------YOUR HISTORY--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        dbc.Col([ dbc.Card([html.H3(children='YOUR HISTORY',style={"color" : "#0F7D91",'font-family': "'Xanh Mono', monospace"}),
        dbc.Col([
             dbc.Table(
                [
                     html.Thead(
                     html.Tr([html.Th(col['name']) for col in columns])
                        ),
                        html.Tbody(id='table-body')
                                ],
                id='table',
                bordered=True,
                hover=True,
                responsive=True,
                striped=True
            )

    ]),
dbc.Pagination(id='pagination', max_value=15,active_page=0,first_last=True,previous_next=True , style={'color': 'red','background-color': 'blue'}),
],className='mt-4',style={"height":"26rem","border-width": "medium",'background':'linear-gradient(to bottom, #B5D3DD, #E1F1F6)','display': 'flex','justify-content': 'center','align-items': 'center'}),], width = 8), ],className = '',justify='center'),
#--------------------------------------------------------------------RECOMMENDED FOR YOU------------------------------------------------------------------------------------------------------------------
    dbc.Row([
    dbc.Card(
    [html.H1(children='RECOMMENDED FOR YOU',style={"color" : "#0F7D91",'font-family': "'Xanh Mono', monospace"}),
    dbc.Col([
        dbc.Label([
        dbc.Label("Enter number of recommended movie:",style={"color" : "#0F7D91",'font-family': "'Xanh Mono', monospace","font-size": "20px","font-weight": "bold"}),
        dbc.Input(id='input_k_rec', type='number', value=5,min=1),
    ]),
         dbc.Table(
            [
            html.Thead([
                html.Tr([
                    html.Th('Title'),
                    html.Th('Genres'),
                    html.Th('Rating'),
                ]),
            ]),
    html.Tbody(id='table_body_rec'),
                            ],
            id='table_rec',
            bordered=True,
            hover=True,
            responsive=True,
            striped=True
        )

    ]),
    dbc.Pagination(id='pagination_rec', max_value=0,active_page=0,first_last=True,previous_next=True),

    ],className='mt-4',style={"height":"26rem","border-width": "medium" ,'background':'linear-gradient(to bottom, #B5D3DD, #E1F1F6)','display': 'flex',
        'justify-content': 'center',
        'align-items': 'center'}),
    ],className = '',justify='center'),
#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
]),

#-----------------------------------------------------------------------------Tab2---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
tab2_content = dbc.Container([
    dbc.Row([dbc.Card([
            dbc.Label (
                [
                    html.H3("Enter a movie title:" ,style={"color" : "#0F7D91",'font-family': "'Xanh Mono', monospace","font-size": "20px","font-weight": "bold"}),
                    dcc.Dropdown(
                        id='input_movie_title',
                        value='',
                        placeholder='Search for a movie...',
                        options=search_Movies,
                    ),
                ]
)],className='my-5',
    style={'display': 'flex', 'flex-direction': 'column', 'justify-content': 'center', 'align-items': 'center' , 'background':'linear-gradient(to bottom, #B5D3DD, #E1F1F6)'}),
            ]),
#-----------------------------------------------------------------------------------------------similar movies-------------------------------------------------------------------------------------------------------------------------------------------------------------
dbc.Row([dbc.Card([
        html.H3("Enter  number to display similar movies ",style={"color" : "#0F7D91",'font-family': "'Xanh Mono', monospace"}),
        dcc.Input(id="input_k", type='number', placeholder="Enter k" , value='0',min=0)],className='my-5',
    style={'display': 'flex', 'flex-direction': 'column', 'justify-content': 'center', 'align-items': 'center','background':'linear-gradient(to bottom, #B5D3DD, #E1F1F6)'}),]),
#---------------------------------------------------------
dbc.Row([dbc.Card([html.H1(children='Similar movies',style={"color" : "#0F7D91",'font-family': "'Xanh Mono', monospace"}),
        html.Div([
            dbc.Table(
                [html.Thead(html.Tr([html.Th(col['name']) for col in columns_movie])),
                 html.Tbody(id='output_div')],
                id='table',
                bordered=True,
                hover=True,
                responsive=True,
                striped=True,
                style={"max-height": "300px", "overflow-y": "auto"}
            )
        ], style={"max-height": "300px", "overflow-y": "auto"})],className='my-5',
style={'display': 'flex', 'flex-direction': 'column', 'justify-content': 'center',
       'align-items': 'center','background':'linear-gradient(to bottom, #B5D3DD, #E1F1F6)'}),]),






#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
]),
#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
tabs = dbc.Tabs(
    [
        dbc.Tab(tab1_content, label='Login' , tab_style={"marginLeft": "auto"} ,label_style={"color" : "#0F7D91",'font-family': "'Xanh Mono', monospace","font-size": "20px","font-weight": "bold"}),
        dbc.Tab(tab2_content, label='Movie' ,label_style={"color" : "#0F7D91",'font-family': "'Xanh Mono', monospace","font-size": "20px","font-weight": "bold"}),
    ]
)
#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
app.layout = html.Div([
    # Navbar
# dbc.Navbar([
#     dbc.Container(
#         [   html.H1('Movie_lens' ,style={"font-family":"Lucida Console"}),
#         ],className='d-flex align-items-center justify-content-center',style={'font-weight': 'bold', 'font-size': '24px'}
#         ),
#     dbc.NavItem(dbc.NavLink(html.Img(src="https://www.svgrepo.com/show/35001/github.svg", height="50px",style={ 'margin-right' : '4rem'}), active=True, href="https://github.com/radwaayman22/-Visualization_Project-ITI_9_Month_Intake43")),],color='#FFFFFF', light=True),
    navbar , tabs
], style={ 'background':'#F7F7F7'}
)

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------Define callbacks---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
@app.callback(
    Output('table-body', 'children'),
    Output('pagination', 'active_page'),
    Input('search-button', 'n_clicks'),
    Input('pagination', 'active_page'),
    State('search-bar', 'value'),
)
def update_table(n_clicks, active_page, user_id):
    if user_id is None:
        return [] ,0

    user_movies = ratings_df[ratings_df['userId'] == int(user_id)]
    merged_movies = pd.merge(user_movies, movies_df, on='movieId')

    total_pages = (len(merged_movies) - 1) // items_per_page + 1
    avg_ratings = merged_movies.groupby(['title', 'genres'])['rating'].mean().reset_index()
    top_movies = avg_ratings.sort_values('rating', ascending=False)

    start_idx = (active_page - 1) * items_per_page
    end_idx = active_page * items_per_page

    paginated_movies = top_movies.iloc[start_idx:end_idx]

    rows = []
    for _, row in paginated_movies.iterrows():
        rows.append(html.Tr([
            html.Td(row['title']),
            html.Td(row['genres']),
            html.Td(row['rating']),
        ]))

    pagination = dbc.Pagination(
        id='pagination',
        max_value=total_pages - 1,
        active_page=active_page,
        first_last=True,
        previous_next=True,
        style={'color': 'red','background-color': 'blue'}
    )

    return rows , active_page

#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
@app.callback(
    Output('table_body_rec', 'children'),
    Output('pagination_rec', 'active_page'),
    Input('search-button', 'n_clicks'),
    Input("input_k_rec", "value"),
    Input('pagination_rec', 'active_page'),
    State('search-bar', 'value'),
)
def update_table(n_clicks,input_k_rec, active_page, search_bar_value):
    if not search_bar_value:
        return [],0

    df_user = predictions_df[predictions_df['userId'] == int(search_bar_value)]
    df_user_sorted = df_user.sort_values(by='prediction', ascending=False)
    df_top_k = df_user_sorted.head(input_k_rec)
    movie_id_to_title = dict(zip(movies_df['movieId'], movies_df['title']))
    top_k_movie_titles = [movie_id_to_title.get(movie_id) for movie_id in df_top_k['movieId']]
    df_top_k_movie_titles = pd.DataFrame({'title': top_k_movie_titles , })
    movie_ids = df_top_k['movieId'].tolist()
    movie_genres = [movies_df[movies_df['movieId'] == movie_id]['genres'].iloc[0] for movie_id in movie_ids]
    movie_avg_ratings = [ratings_df[ratings_df['movieId'] == movie_id]['rating'].mean() for movie_id in movie_ids]
    df_top_k_movie_titles['genres'] = movie_genres
    df_top_k_movie_titles['average_rating'] = movie_avg_ratings

    total_pages = (len(df_top_k_movie_titles) - 1) // items_per_page + 1

    start_idx = (active_page - 1) * items_per_page
    end_idx = active_page * items_per_page

    paginated_movies = df_top_k_movie_titles.iloc[start_idx:end_idx]


    rows_rec = []
    for _, row in paginated_movies.iterrows():
        rows_rec.append(html.Tr([
            html.Td(row['title']),
            html.Td(row['genres']),
            html.Td(row['average_rating']),
        ]))

    pagination = dbc.Pagination(
        id='pagination_rec',
        max_value=total_pages - 1,
        active_page=active_page ,
        first_last=True,
        previous_next=True,
    )

    return rows_rec , active_page

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
@app.callback(
    Output(component_id="output_div", component_property="children"),
    [Input(component_id="input_k", component_property="value"),
     State(component_id="input_movie_title", component_property="value")]
)
def update_output(input_k, input_movie_title):
    if not input_movie_title:
        return []
    table_rows = []
    top_k_movies = get_similar_movies(input_movie_title, int(input_k))
    num_rows = min(int(input_k), len(top_k_movies))
    table_rows = [
        html.Tr([html.Td(top_k_movies.iloc[i]["title"]), html.Td(top_k_movies.iloc[i]["genres"]), html.Td(top_k_movies.iloc[i]["average_rating"])])
        for i in range(num_rows)
    ]
    return table_rows

#------------------------------------------------------------Run-server--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
